In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2

In [7]:
%matplotlib inline
import sys
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *

LANDSCAPE_TYPES ={"RNA": [0, 1], 'TF':['SIX6_REF_R1']} # for testing 
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
DNA_alphabet = "TCGA"
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


B1L14RNA1 loaded


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## DQN Explorer

In [8]:
from explorers.dqn_explorer import DQN_Explorer 

num_of_rounds = 5

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(train_epochs=10)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.14554841726711917
round: 1, cost: 1, evals: 0, top: 0.14554841726711917
round: 2, cost: 1, evals: 0, top: 0.14554841726711917
round: 3, cost: 1, evals: 0, top: 0.14554841726711917
round: 4, cost: 1, evals: 0, top: 0.14554841726711917


In [15]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object = next(landscape_generator)
wt = landscape_object['starting_seqs']['TF0']
landscape_oracle = landscape_object['landscape_oracle']

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=DNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(alphabet=DNA_alphabet, train_epochs=10, virtual_screen=20)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

1 TF landscapes loaded.


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.6781429101554447
round: 1, cost: 1, evals: 0, top: 0.6781429101554447
round: 2, cost: 1, evals: 0, top: 0.6781429101554447
round: 3, cost: 1, evals: 0, top: 0.6781429101554447
round: 4, cost: 1, evals: 0, top: 0.6781429101554447


In [9]:
from explorers.dqn_explorer import DQN_Explorer 

dqn_explorer_prod = DQN_Explorer(train_epochs=10)
dqn_explorer_prod.debug = False

In [ ]:
# run all evaluations 
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=3)

loading landscapes RNA: [0, 1], TF:['SIX6_REF_R1']
B1L14RNA1 loaded
B1L50RNA1 loaded
1 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0


In [10]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.consistency_robustness_independence, num_starts=3)

loading landscapes RNA: [0, 1], TF:['SIX6_REF_R1']
B1L14RNA1 loaded
B1L50RNA1 loaded
1 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0
round: 0, cost: 1, evals: 0, top: 0.5437468768175177


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


round: 1, cost: 1, evals: 0, top: 0.5437468768175177
round: 2, cost: 1, evals: 0, top: 0.5437468768175177
round: 3, cost: 1, evals: 0, top: 0.5437468768175177
round: 4, cost: 1, evals: 0, top: 0.5437468768175177
round: 5, cost: 1, evals: 0, top: 0.5437468768175177
round: 6, cost: 1, evals: 0, top: 0.5437468768175177
round: 7, cost: 1, evals: 0, top: 0.5437468768175177
round: 8, cost: 1, evals: 0, top: 0.5437468768175177
round: 9, cost: 1, evals: 0, top: 0.5437468768175177
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 101, evals: 843, top: 0.6041632197437043
round: 2, cost: 201, evals: 1600, top: 0.6892953202744071
round: 3, cost: 301, evals: 2155, top: 0.6892953202744071
round: 4, cost: 398, evals: 2647, top: 0.6892953202744071
round: 5, cost: 495, evals: 2904, top: 0.6892953202744071
round: 6, cost: 589, evals: 3162, top: 0.6892953202744071
round: 7, cost: 685, evals: 3417, top: 0.6892953202744071
round: 8, cost: 783, evals: 3655, top: 0.6892953202744071
round: 

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.5437468768175177
[0.527049137548655, 0.0012565373575499437, 0.47169432509379505]
round: 1, cost: 101, evals: 137.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 102, evals: 138.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 3, cost: 102, evals: 138.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 4, cost: 102, evals: 138.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 5, cost: 102, evals: 138.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 6, cost: 102, evals: 138.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 7, cost: 102, evals: 138.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 

...

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 3, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 4, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 5, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 6, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 7, cost: 1, evals: 0.0, top: 0.6781429101554447
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 8, c

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.5502102832639261
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 101, evals: 777.0, top: 0.9233084566857709
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 201, evals: 1470.0, top: 0.9233084566857709
[0.28381878500135016, 0.13534634743678528, 0.5808348675618645]
round: 3, cost: 292, evals: 2045.0, top: 0.9604275759699832
[0.008410638682307637, 0.9831787226353847, 0.008410638682307637]
round: 4, cost: 379, evals: 2436.0, top: 0.9604275759699832
[0.37666407267810237, 0.6208936247857315, 0.0024423025361661643]
round: 5, cost: 464, evals: 2668.0, top: 0.9604275759699832
[0.9875703946823301, 0.006214802658834955, 0.006214802658834955]
round: 6, cost: 545, evals: 2895.0, top: 0.9604275759699832
[0.38816063592327554, 0.3044431557059038, 0.30739620837082066]
round: 7, cost: 636, evals: 3067.0, top: 0.9604275759699832
[0.7140192440692719, 0

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.33058796849874245
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 101, evals: 790.0, top: 0.864831154908671
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 201, evals: 1451.0, top: 0.8750566940172351
[0.6444700656498236, 0.3519455405927196, 0.003584393757456888]
round: 3, cost: 299, evals: 1952.0, top: 0.9804354100523647
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 4, cost: 392, evals: 2315.0, top: 0.9804354100523647
[0.5000498268427849, 0.11260514809006114, 0.38734502506715396]
round: 5, cost: 471, evals: 2537.0, top: 0.9804354100523647
[0.004739096761045271, 0.9905218064779094, 0.004739096761045271]
round: 6, cost: 562, evals: 2760.0, top: 0.9804354100523647
[0.35084982770248097, 0.21263415837426175, 0.4365160139232573]
round: 7, cost: 643, evals: 2923.0, top: 0.9804354100523647
[0.3946962959042748, 0.33570

## BO Explorer

In [ ]:
%autoreload 2
from explorers.bo_explorer import BO_Explorer

num_of_rounds = 5
arch1 = NLNN(len(wt), hidden_dims=10, epochs=20)
arch2 = NLNN(len(wt), hidden_dims=20, epochs=20)
arch3 = NLNN(len(wt), hidden_dims=30, epochs=20)
arch4 = NLNN(len(wt), hidden_dims=40, epochs=20)
# ensemble we will use for BO Explorer
nn_model_1 = NN_model(landscape_oracle, arch1)
nn_model_2 = NN_model(landscape_oracle, arch2)
nn_model_3 = NN_model(landscape_oracle, arch3)
nn_model_4 = NN_model(landscape_oracle, arch4)
models = [nn_model_1, nn_model_2, nn_model_3, nn_model_4]
ensemble_model = Ensemble_models(models)
ensemble_model.update_model(training_sequences)

In [ ]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer()
bo_explorer.debug = True
bo_explorer.set_model(ensemble_model)
bo_explorer.run(num_of_rounds, verbose=True)

In [ ]:
from explorers.bo_explorer import BO_Explorer

bo_explorer_prod = BO_Explorer()
bo_explorer_prod.debug = False

In [ ]:
# run all evaluations 
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.scalability, num_starts=3)

In [ ]:
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.consistency_robustness_independence, num_starts=3)